In [1]:
import cv2
import os, glob
import mediapipe as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
import random

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, ConvLSTM2D, Dropout, LSTM, Rescaling

In [4]:
orig_img_targ = r'D:\datasets\Celeb_df_as_imgs\Original'
fake_img_targ = r'D:\datasets\Celeb_df_as_imgs\Fake'

In [5]:
orig_dir = r'D:\datasets\Celeb-DF\Celeb-real'
fake_dir = r'D:\datasets\Celeb-DF\Celeb-synthesis'
orig_dir_yt = r'D:\datasets\Celeb-DF\YouTube-real'

In [6]:
orig_files = glob.glob(os.path.join(orig_dir, '*mp4'))
fake_files = glob.glob(os.path.join(fake_dir, '*mp4'))

In [7]:
orig_files_yt = glob.glob(os.path.join(orig_dir_yt, '*mp4'))

In [8]:
print(len(orig_files))
print(len(fake_files))
print(len(orig_files_yt))

158
795
250


In [9]:
orig_files = orig_files+orig_files_yt

In [10]:
print(len(orig_files))
print(len(fake_files))

408
795


In [11]:
trial_orig = orig_files[:200]
trial_fakes = fake_files[:200]

In [12]:
def extract_faces_from_video(video_path, output_directory='output_faces'):
    # Initialize MediaPipe Face Detection
    mp_face_detection = mp.solutions.face_detection
    face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

    # Open the video file
    video_capture = cv2.VideoCapture(video_path)

    # Check if the video file opened successfully
    if not video_capture.isOpened():
        print("Error: Unable to open video file.")
        return None

    # Create a directory to save the face images
    # os.makedirs(output_directory, exist_ok=True)
    # path_list = []
    # Loop through each frame of the video
    frames = []
    
    frame_count = 0
    while True:
        # Read the current frame
        ret, frame = video_capture.read()

        # If there are no more frames, break out of the loop
        if not ret:
            break

        # Convert the frame to RGB (MediaPipe requires RGB input)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Perform face detection
        results = face_detection.process(frame_rgb)
        if len(frames) <10:
        # Draw bounding boxes around the detected faces
            if results.detections:
                for detection in results.detections:
                    # Get the bounding box coordinates
                    bboxC = detection.location_data.relative_bounding_box
                    ih, iw, _ = frame.shape
                    x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
                                 int(bboxC.width * iw), int(bboxC.height * ih)
    
                    # Calculate expanded bounding box dimensions
                    head_width = int(w * 1.5)  # 50% wider
                    head_height = int(h * 1.5)  # 100% taller
    
                    # Adjust bounding box position to ensure it remains within frame boundaries
                    x = max(0, x - (head_width - w) // 2)
                    y = max(0, y - (head_height - h) // 2)
                    head_width = min(iw - x, head_width)
                    head_height = min(ih - y, head_height)
    
                    # Crop the bounding box from the frame
                    face_image = frame[y:y + head_height, x:x + head_width]
                    face_image = cv2.resize(face_image, (80, 80))
                    # Save the cropped face image
                    # face_filename = os.path.join(output_directory, f'face_{frame_count}.jpg')
                    # cv2.imwrite(face_filename, face_image)
                    # path_list.append(face_filename)
                    frames.append(face_image)
        
                # Increment the frame count for naming the next face image
        frame_count += 1

    # Release the video capture object and close all windows
    video_capture.release()
    cv2.destroyAllWindows()
    print("frames = ", frame_count)
    # return output_directory
    return frames


In [13]:
a = extract_faces_from_video(trial_fakes[0])

frames =  469


In [14]:
print(a)

[array([[[  0,   0,  60],
        [  0,   0,  62],
        [  0,   1,  66],
        ...,
        [ 50,  78, 121],
        [ 86, 119, 148],
        [112, 147, 172]],

       [[  0,   0,  60],
        [  0,   0,  62],
        [  0,   1,  66],
        ...,
        [ 10,  31,  98],
        [ 58,  88, 129],
        [121, 154, 185]],

       [[  0,   0,  61],
        [  0,   2,  63],
        [  0,   3,  68],
        ...,
        [  9,  25, 123],
        [ 57,  82, 142],
        [124, 156, 193]],

       ...,

       [[ 17,  33,  46],
        [ 14,  30,  43],
        [ 24,  40,  53],
        ...,
        [ 12,  29,  37],
        [ 12,  29,  37],
        [ 12,  29,  37]],

       [[ 10,  25,  36],
        [ 10,  25,  36],
        [ 10,  25,  36],
        ...,
        [ 11,  28,  37],
        [ 11,  28,  37],
        [ 11,  28,  37]],

       [[ 10,  22,  32],
        [ 10,  22,  32],
        [ 10,  22,  32],
        ...,
        [ 11,  28,  37],
        [ 11,  28,  37],
        [ 11,  28,  37]

In [15]:
trial_orig_video_frames = []

for i in range(150, 200):
    frames_i = extract_faces_from_video(trial_orig[i])
    trial_orig_video_frames.append(frames_i)
    print(i , ' done')

frames =  364
150  done
frames =  492
151  done
frames =  362
152  done
frames =  420
153  done
frames =  401
154  done
frames =  456
155  done
frames =  461
156  done
frames =  314
157  done
frames =  450
158  done
frames =  415
159  done
frames =  512
160  done
frames =  295
161  done
frames =  450
162  done
frames =  463
163  done
frames =  486
164  done
frames =  474
165  done
frames =  477
166  done
frames =  575
167  done
frames =  422
168  done
frames =  470
169  done
frames =  397
170  done
frames =  474
171  done
frames =  461
172  done
frames =  442
173  done
frames =  319
174  done
frames =  282
175  done
frames =  435
176  done
frames =  468
177  done
frames =  456
178  done
frames =  462
179  done
frames =  471
180  done
frames =  463
181  done
frames =  547
182  done
frames =  470
183  done
frames =  499
184  done
frames =  481
185  done
frames =  473
186  done
frames =  463
187  done
frames =  472
188  done
frames =  510
189  done
frames =  463
190  done
frames =  461
19

In [16]:
for i in range(len(trial_orig_video_frames)):
    if len(trial_orig_video_frames[i]) !=10:
        print(i, len(trial_orig_video_frames[i]))

41 4


In [17]:
trial_orig_video_frames.pop(41)

[array([[[151, 166, 173],
         [153, 166, 174],
         [153, 166, 174],
         ...,
         [146, 156, 155],
         [146, 156, 155],
         [144, 154, 153]],
 
        [[142, 165, 171],
         [147, 165, 172],
         [147, 165, 172],
         ...,
         [142, 159, 157],
         [138, 155, 153],
         [139, 156, 154]],
 
        [[145, 164, 171],
         [145, 165, 172],
         [145, 165, 172],
         ...,
         [142, 159, 157],
         [138, 155, 153],
         [135, 157, 154]],
 
        ...,
 
        [[ 47,  52,  50],
         [ 56,  91,  84],
         [ 50,  76,  70],
         ...,
         [ 62,  61,  59],
         [ 47,  68,  63],
         [ 92, 133, 124]],
 
        [[ 48,  51,  49],
         [ 62,  83,  78],
         [ 53,  77,  71],
         ...,
         [ 65,  57,  57],
         [ 53,  71,  66],
         [ 39,  69,  62]],
 
        [[ 62,  57,  56],
         [ 55,  61,  59],
         [ 62,  62,  60],
         ...,
         [ 67,  59,  59],
  

In [18]:
trial_orig_video_frames = np.array(trial_orig_video_frames)

In [19]:
trial_orig_video_frames.shape

(49, 10, 80, 80, 3)

In [20]:
trial_fake_video_frames = []

for i in range(150,200):
    frames_i = extract_faces_from_video(trial_fakes[i])
    trial_fake_video_frames.append(frames_i)
    print(i , ' done')

frames =  303
150  done
frames =  410
151  done
frames =  345
152  done
frames =  224
153  done
frames =  284
154  done
frames =  315
155  done
frames =  325
156  done
frames =  272
157  done
frames =  308
158  done
frames =  311
159  done
frames =  317
160  done
frames =  316
161  done
frames =  363
162  done
frames =  303
163  done
frames =  410
164  done
frames =  345
165  done
frames =  224
166  done
frames =  284
167  done
frames =  315
168  done
frames =  325
169  done
frames =  272
170  done
frames =  308
171  done
frames =  311
172  done
frames =  317
173  done
frames =  316
174  done
frames =  363
175  done
frames =  303
176  done
frames =  410
177  done
frames =  345
178  done
frames =  224
179  done
frames =  284
180  done
frames =  315
181  done
frames =  325
182  done
frames =  272
183  done
frames =  308
184  done
frames =  311
185  done
frames =  317
186  done
frames =  316
187  done
frames =  363
188  done
frames =  303
189  done
frames =  410
190  done
frames =  345
19

In [21]:
for i in range(len(trial_fake_video_frames)):
    if len(trial_fake_video_frames[i]) !=10:
        print(i, len(trial_fake_video_frames[i]))

In [22]:
trial_fake_video_frames = np.array(trial_fake_video_frames)


In [23]:
trial_fake_video_frames.shape

(50, 10, 80, 80, 3)

In [24]:
trial_fake_video_frames = np.concatenate((trial_fake_video_frames, trial_orig_video_frames), axis=0)

In [25]:
trial_fake_y = np.array([0 for i in range(49)])
trial_orig_y = np.array([1 for i in range(50)])

In [26]:
trial_fake_y  = np.concatenate((trial_fake_y, trial_orig_y), axis=0)

In [27]:
file_path = 'X_test.npy'

np.save(file_path, trial_fake_video_frames)

In [28]:
file_path = 'Y_test.npy'

np.save(file_path, trial_fake_y)